In [14]:
NUM_EPOCHS = 100
BATCH_SIZE = 16
MOMENTUM = 0.9
LR_DECAY = 0.0001
LR_INIT = 0.005
IMAGE_DIM = 227
NUM_CLASSES = 5
OUTPUT_PATH = "/content/drive/MyDrive/Projects/Cricket07/model.pth"

In [15]:
import numpy as np
FILE_NAME = "/content/drive/MyDrive/Projects/Cricket07/training_data.npz"

data = np.load(FILE_NAME)
X = data['X']
X = X[:, np.newaxis, :, :]
Y = data['Y']

In [16]:
import torch
from torch.utils.data import TensorDataset, DataLoader

tensor_x = torch.Tensor(X)
tensor_y = torch.Tensor(Y)

my_dataset = TensorDataset(tensor_x, tensor_y)
trainloader = DataLoader(my_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [17]:
import torch
import torch.nn as nn

class AlexNet(nn.Module):
    def __init__(self, in_channels=1, num_classes=NUM_CLASSES):
        super(AlexNet, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=96, kernel_size=11, stride=4),  # (8 x 96 x 55 x 55)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (8 x 96 x 27 x 27)
            nn.Conv2d(96, 256, 5, padding=2),  # (8 x 256 x 27 x 27)
            nn.ReLU(),
            nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (8 x 256 x 13 x 13)
            nn.Conv2d(256, 384, 3, padding=1),  # (8 x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding=1),  # (8 x 384 x 13 x 13)
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding=1),  # (8 x 256 x 13 x 13)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),  # (8 x 256 x 6 x 6)
        )
        self.linear = nn.Sequential(
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=(256 * 6 * 6), out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=num_classes),
        )

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 256 * 6 * 6)
        return self.linear(x)

# Example usage:
# model = AlexNet()
# x = torch.randn(1, 1, 227, 227)
# print(model(x).shape)

In [18]:
for img, cls in trainloader:
    print(img.shape)
    break

torch.Size([16, 1, 227, 227])


In [ ]:
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.autograd.set_detect_anomaly(True)

alexnet = AlexNet(num_classes=NUM_CLASSES).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)

print('Starting training...')

for epoch in tqdm(range(NUM_EPOCHS)):

    correct = 0
    total_samples = 0
    for imgs, classes in trainloader:
        imgs, classes = imgs.to(device), classes.to(device)

        optimizer.zero_grad()

        op = alexnet(imgs)
        output = op.clone()
        loss = F.cross_entropy(output, classes)

        loss.backward()
        optimizer.step()

        correct += (output.argmax(dim=1) == classes.argmax(dim=1)).sum().item()
        total_samples += classes.size(0)

    accuracy = (correct / total_samples) * 100
    print(f'Epoch {epoch + 1}: Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

print("Finished training")
torch.save(alexnet.state_dict(), OUTPUT_PATH)

Starting training...


  1%|          | 1/100 [00:03<05:59,  3.63s/it]

Epoch 1: Loss: 1.6036, Accuracy: 16.85%


  2%|▏         | 2/100 [00:06<05:38,  3.46s/it]

Epoch 2: Loss: 1.5884, Accuracy: 19.49%


...

Epoch 96: Loss: 0.7174, Accuracy: 76.00%


 97%|█████████▋| 97/100 [05:19<00:09,  3.25s/it]

Epoch 97: Loss: 0.2586, Accuracy: 75.51%


 98%|█████████▊| 98/100 [05:23<00:06,  3.21s/it]

Epoch 98: Loss: 0.6360, Accuracy: 75.51%
